In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.optim as optim
from sklearn.model_selection import KFold
import numpy as np
import itertools
import os
import pickle

replications = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'

np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)


In [ ]:
class Net(nn.Module):
  def __init__(self, num_layers, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_size)])
    self.hidden_layers.extend([nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 2)])
    self.output_layer = nn.Linear(hidden_size, output_size)
    self.activation = nn.ReLU()

  def forward(self, x):
    for layer in self.hidden_layers:
      x = self.activation(layer(x))
    x = self.output_layer(x)
    return x

def make_training_fn(model, loss_fn, optimizer):
  def training_step(x, y):
    model.train()

    yhat = model(x)
    loss = loss_fn(yhat, y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()
  return training_step

def make_val_fn(model, loss_fn):
  def val_step(x, y):
    model.eval()

    with torch.no_grad():
      yhat = model(x)
      loss = loss_fn(yhat, y)
    return loss.item()
  return val_step

def mini_batch_loop(device, data_loader, step_fn):
  mini_batch_losses = []
  for x, y in data_loader:
    x_batch = x.to(device)
    y_batch = y.unsqueeze(-1).to(device)
    mini_batch_losses.append(step_fn(x_batch, y_batch))
  return np.mean(mini_batch_losses)

def cross_validation(train_data, num_layers, input_size, hidden_size, output_size, weight_decay, rep_seed, K=10, repeats=1, epochs=10):
  repeatCV = []
  for r in range(repeats):
    # Split data into folds
    folds = KFold(n_splits=K, shuffle=True, random_state=rep_seed)
    val_losses = []
    for fold, (train_idx, val_idx) in enumerate(folds.split(train_data)):
      train_subset = Subset(train_data, train_idx)
      val_subset = Subset(train_data, val_idx)

      train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
      val_loader = DataLoader(val_subset, batch_size=64, shuffle=True)

      # Instantiate model
      model = Net(num_layers, input_size, hidden_size, output_size).to(device)

      # Instantiate optimizer
      optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay)
      loss_fn = nn.MSELoss()

      # Instantiate training and validation functions
      train_step_fn = make_training_fn(model, loss_fn, optimizer)
      val_step_fn = make_val_fn(model, loss_fn)

      for epoch in range(epochs):
        mini_batch_loop(device, train_loader, train_step_fn)
      # Evaluate at the end of the epochs
      val_losses.append(mini_batch_loop(device, val_loader, val_step_fn))
    repeatCV.append(np.mean(val_losses))
  return np.mean(repeatCV)

def data_splitting(train_data, num_layers, input_size, hidden_size, output_size, weight_decay, rep_seed, epochs=10):
  train_data, val_data = random_split(train_data, [0.8, 0.2], generator=torch.Generator().manual_seed(rep_seed))
  train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
  val_loader = DataLoader(val_data, batch_size=128, shuffle=True)
  # Instantiate model
  model = Net(num_layers, input_size, hidden_size, output_size).to(device)
  # Instantiate optimizer
  optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay)
  loss_fn = nn.MSELoss()

  # Instantiate training and validation functions
  train_step_fn = make_training_fn(model, loss_fn, optimizer)
  val_step_fn = make_val_fn(model, loss_fn)

  for epoch in range(epochs):
    mini_batch_loop(device, train_loader, train_step_fn)

  # Evaluate at the end of the epochs
  val_loss = mini_batch_loop(device, val_loader, val_step_fn)
  return val_loss

In [ ]:
Sigma = np.eye(10)
Sigma[2, 3] = Sigma[3, 2] = 0.5
Sigma[4, 5] = Sigma[5, 4] = -0.5
Sigma[6, 7] = Sigma[7, 6] = 0.2
Sigma[8, 9] = Sigma[9, 8] = 0.9
mu = np.random.rand(10)

def generate_data(n, Sigma, mu):
  X = np.random.multivariate_normal(mu, Sigma, n)
  eps = np.random.normal(0, 5, n)

  y = 0.5*X[:, 0] + 0.75*X[:, 1]**2 - 0.3*np.sin(X[:, 2]) - 0.5*np.abs(X[:, 3])**(1/2)*X[:, 1] + X[:, 4]*np.exp(X[:, 5]) + 1.0*X[:, 6]*np.cos(X[:, 7]) + -0.25*X[:, 8]*X[:, 9]**4 + eps
  return X, y

class DataFrameDataset(Dataset):
  def __init__(self, X_tensor, y_tensor):
    self.X = X_tensor
    self.y = y_tensor

  def __getitem__(self, index):
    return (self.X[index], self.y[index])

  def __len__(self):
    return len(self.X)


In [ ]:
num_layers = [4, 5, 6, 7]
hidden_size = [10, 20, 30]
weight_decay = [0.001, 0.01, 0.1]

hyperparameter_combos = itertools.product(num_layers, hidden_size, weight_decay)

In [ ]:
X_train, y_train = generate_data(10000, Sigma, mu)
X_test, y_test = generate_data(1000000, Sigma, mu)
train_data = DataFrameDataset(torch.as_tensor(X_train).float(), torch.as_tensor(y_train).float())
test_data = DataFrameDataset(torch.as_tensor(X_test).float(), torch.as_tensor(y_test).float())

train_data_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=256)

input_size = train_data.X.shape[1]
output_size = 1
num_epochs = 25

In [ ]:
result_file = './drive/MyDrive/chosen_files/true_result.pkl'
if not os.path.exists(result_file):
  val_losses = []
  for nl, hs, wd in hyperparameter_combos:
    # Instantiate model
    model = Net(nl, input_size, hs, output_size).to(device)

    # Instantiate optimizer
    optimizer = optim.Adam(model.parameters(), weight_decay=wd)
    loss_fn = nn.MSELoss()

    # Instantiate training and validation functions
    train_step_fn = make_training_fn(model, loss_fn, optimizer)
    val_step_fn = make_val_fn(model, loss_fn)

    for epoch in range(num_epochs):
      mini_batch_loop(device, train_data_loader, train_step_fn)

    val_loss = mini_batch_loop(device, test_data_loader, val_step_fn)
    val_losses.append(val_loss)

  with open(result_file, 'wb') as file:
    pickle.dump(val_losses, file)

print(np.argmin(val_losses))

33


In [ ]:
# Data splitting
for i in range(replications):
  result_file = f'./drive/MyDrive/chosen_files/data_split_result_{i}.pkl'
  if not os.path.exists(result_file):
    print(f'Replication {i}')
    chosen_val = []
    losses = []
    hyperparameter_combos = itertools.product(num_layers, hidden_size, weight_decay)
    for nl, hs, wd in hyperparameter_combos:
      val_loss = data_splitting(train_data, nl, train_data.X.shape[1], hs, output_size, wd, i, num_epochs)
      losses.append(val_loss)
    chosen_val.append(np.argmin(losses))

    with open(result_file, 'wb') as file:
        pickle.dump(chosen_val, file)

In [ ]:
# Repeat Kfold CV
for i in range(replications):
  result_file = f'./drive/MyDrive/chosen_files/repeatcv_result_{i}.pkl'
  if not os.path.exists(result_file):
    print(f'Replication {i}')
    chosen_val = []
    losses = []
    hyperparameter_combos = itertools.product(num_layers, hidden_size, weight_decay)
    for nl, hs, wd in hyperparameter_combos:
      val_loss = cross_validation(train_data, nl, train_data.X.shape[1], hs, output_size, wd, i, 5, 5, num_epochs)
      losses.append(val_loss)
    chosen_val.append(np.argmin(losses))

    with open(result_file, 'wb') as file:
        pickle.dump(chosen_val, file)

In [ ]:
# Kfold CV
for i in range(replications):
  result_file = f'./drive/MyDrive/chosen_files/kfoldcv_result_{i}.pkl'
  if not os.path.exists(result_file):
    print(f'Replication {i}')
    chosen_val = []
    losses = []
    hyperparameter_combos = itertools.product(num_layers, hidden_size, weight_decay)
    for nl, hs, wd in hyperparameter_combos:
      val_loss = cross_validation(train_data, nl, train_data.X.shape[1], hs, output_size, wd, i, 5, 1, num_epochs)
      losses.append(val_loss)
    chosen_val.append(np.argmin(losses))

    with open(result_file, 'wb') as file:
        pickle.dump(chosen_val, file)